In [1]:
!pip install dagviz
import dagviz
from dagviz import render_svg
from dagviz.style.metro import svg_renderer, StyleConfig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import networkx as nx

In [3]:
def degree_seq(adj):
    return np.sum(adj, axis=0)

def deg_assortativity(adj):
    # Calculations based on PHYSICAL REVIEW E 84, 047101 (2011)
    n = len(adj)
    m = np.sum(adj)/2.0
    deg = degree_seq(adj)
    M2, di1, di2 = 0, 0, 0
    for i in range(n-1):
        for j in range(i+1, n):
            if adj[i, j] == 1:
                M2 += deg[i] * deg[j]
                di1 += (deg[i] + deg[j])/2.0
                di2 += (deg[i]**2 + deg[j]**2)/2.0
    #di2 = ((np.sum(deg**2)/2)**2)/m
    #return (M2 - di2) / ((np.sum(deg**3)/2) - di2)
    di1 = (di1/m) ** 2
    di2 /= m
    M2 /= m
    return (M2 - di1)/(di2 - di1)

def switch_4e(adj, e1, e2, e3, e4):
    mat = np.copy(adj)
    mat[e1[0], e1[1]], mat[e1[1], e1[0]], mat[e2[0], e2[1]], mat[e2[1], e2[0]] = 0, 0, 0, 0
    mat[e3[0], e3[1]], mat[e3[1], e3[0]], mat[e4[0], e4[1]], mat[e4[1], e4[0]] = 1, 1, 1, 1
    return mat

def switch_checkerboard(adj, i, j, k, l):
    mat = np.copy(adj)
    mat[i, k], mat[k, i], mat[j, k], mat[k, j] = 1-mat[i, k], 1-mat[k, i], 1-mat[j, k], 1-mat[k, j]
    mat[i, l], mat[l, i], mat[j, l], mat[l, j] = 1-mat[i, l], 1-mat[l, i], 1-mat[j, l], 1-mat[l, j]
    return mat

def is_checkerboard(ik, il, jk, jl):
    if ik == jl and il == jk and ik == 1-jk:
        return True
    else:
        return False

def find_adj(adj, mat_list):
    for i in range(len(mat_list)):
        if np.array_equal(adj, mat_list[i]):
            return i
    return -1

def all_next_switches_XBS(adj):
    adj_r = deg_assortativity(adj)
    next_swt = []
    swt_type = []
    edgeIdxs = np.where(adj==1)
    edgeIdxs = [(edgeIdxs[0][i], edgeIdxs[1][i]) for i in range(len(edgeIdxs[0])) if edgeIdxs[0][i]<edgeIdxs[1][i]]
    for e1_idx in range(len(edgeIdxs)-1):
        for e2_idx in range(e1_idx+1, len(edgeIdxs)):
            e1, e2 = edgeIdxs[e1_idx], edgeIdxs[e2_idx]
            if len(set(e1+e2)) != 4:
                continue
            swt_options = [(e1[0], e2[0], e1[1], e2[1]), (e1[0], e2[1], e1[1], e2[0])]
            for opt in swt_options:
                if (adj[opt[0], opt[1]] == 1) or (adj[opt[2], opt[3]] == 1):
                    continue
                new_adj = switch_4e(adj, e1, e2, (opt[0], opt[1]), (opt[2], opt[3]))
                new_r =  deg_assortativity(new_adj)
                if adj_r < new_r:
                    swt_type.append(True)
                elif adj_r > new_r:
                    swt_type.append(False)
                else:
                    if (opt[0] * opt[1] + opt[2]* opt[3]) - (e1[0] * e1[1] + e2[0] * e2[1]) > 0:
                        swt_type.append(True)
                    else:
                        swt_type.append(False)
                next_swt.append(new_adj)
    return next_swt, swt_type

def all_next_switches(adj):
    adj_r = deg_assortativity(adj)
    n = len(adj)
    next_swt = []
    swt_type = []
    for row1 in range(n-1):
        for row2 in range(row1+1, n):
            for col1 in range(row1+1, n-1):
                if row2 == col1:
                    continue
                for col2 in range(col1+1, n):
                    if row2 == col2:
                        continue
                    if is_checkerboard(adj[row1, col1], adj[row1, col2], adj[row2, col1], adj[row2, col2]):
                        new_adj = switch_checkerboard(adj, row1, row2, col1, col2)
                        new_r =  deg_assortativity(new_adj)
                        if adj[row1, col1] == 0:
                            swt_type.append(True)
                        else:
                            swt_type.append(False)
                        next_swt.append(new_adj)
    return next_swt, swt_type

In [4]:
A = [[0, 0, 1, 1, 1, 1, 0],
     [0, 0, 1, 1, 0, 0, 1],
     [1, 1, 0, 0, 0, 0, 0],
     [1, 1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0, 0]]
A = [[0, 0, 1, 1, 1, 1],
     [0, 0, 1, 1, 1, 0],
     [1, 1, 0, 0, 0, 0],
     [1, 1, 0, 0, 0, 0],
     [1, 1, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0]]

#G=nx.from_numpy_array(np.array(A))
A = np.array(A)

d_space = []
d_space_arcs = []
d_space_r = []
d_space.append(A)
d_space_r.append(deg_assortativity(A))
d_space_pointer = 0

while d_space_pointer < len(d_space):
  nxt_swts, nxt_arcs = all_next_switches(d_space[d_space_pointer])
  for nxtswt_idx in range(len(nxt_swts)):
      nxt_swt = nxt_swts[nxtswt_idx]
      dspace_adj_index = find_adj(nxt_swt, d_space)
      if dspace_adj_index == -1:
          d_space.append(nxt_swt)
          d_space_r.append(deg_assortativity(nxt_swt))
          dspace_adj_index = len(d_space)-1
      if nxt_arcs[nxtswt_idx]:
          if (d_space_pointer, dspace_adj_index) not in d_space_arcs:
              d_space_arcs.append((d_space_pointer, dspace_adj_index))
      else:
          if (dspace_adj_index, d_space_pointer) not in d_space_arcs:
              d_space_arcs.append((dspace_adj_index, d_space_pointer))
  d_space_pointer += 1


In [ ]:
for mat_id in range(len(d_space)):
    mat = d_space[mat_id]
    plt.figure()
    cmap = colors.ListedColormap(['white', 'tab:blue'])
    plt.imshow(mat, cmap=cmap)
    plt.xticks([])
    plt.yticks([])
    #plt.tight_layout()
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    #plt.savefig(f"filename.png", transparent=True, bbox_inches="tight", pad_inches=0)
    plt.savefig('mat_'+str(mat_id)+'_'+str(round(deg_assortativity(mat), 4))+'.pdf', format='pdf', transparent=True, bbox_inches="tight", pad_inches=0)
print(d_space_r)
m = np.sum(A)/2.0
sort_idxes = np.argsort(d_space_r)
for idx in sort_idxes:
    print(idx, d_space_r[idx])

In [ ]:
print(d_space_arcs)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 3), (1, 6), (1, 10), (1, 8), (1, 2), (2, 4), (2, 5), (2, 11), (2, 9), (3, 5), (3, 10), (3, 7), (3, 4), (4, 6), (4, 12), (4, 9), (5, 11), (5, 7), (5, 6), (6, 12), (6, 8), (7, 8), (10, 7), (7, 9), (11, 7), (10, 8), (8, 9), (12, 8), (11, 9), (12, 9), (10, 12), (10, 11), (11, 12)]


In [23]:
G = nx.DiGraph()
for i in range(len(d_space)):
    G.add_node(f"{i}")

for e in d_space_arcs:
    G.add_edge(f"{e[0]}", f"{e[1]}")

dagviz.Metro(G)
r = dagviz.render_svg(G, style=svg_renderer(StyleConfig(label_arrow_dash_array=0, label_font_size=0, scale=20, node_radius=0, arc_radius=25)))
with open("EHS.svg", "wt") as fs:
    fs.write(r)

In [ ]:
plt.figure()
cmap = colors.ListedColormap(['white', 'tab:blue'])
plt.imshow(d_space[1], cmap=cmap)
plt.xticks([])
plt.yticks([])
#plt.tight_layout()
plt.gca().set_axis_off()
plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
plt.margins(0, 0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())
#plt.savefig(f"filename.png", transparent=True, bbox_inches="tight", pad_inches=0)
#plt.savefig('A'+str(round(deg_assortativity(A), 4))+'.pdf', format='pdf', transparent=True, bbox_inches="tight", pad_inches=0)
#plt.cla()
#plt.savefig('A'+str(round(deg_assortativity(mat), 4))+'.pdf', format='pdf')

In [ ]:
plt.gca().set_axis_off()
plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
plt.margins(0, 0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())
plt.savefig(f"filename.png", transparent=True, bbox_inches="tight", pad_inches=0)
plt.cla()